## Dataset: [College Majors (recent-grads)](https://github.com/fivethirtyeight/data/tree/master/college-majors)
Source : [FiveThirtyEight datasets](https://data.fivethirtyeight.com/)


### Analise Burko & Marcos de la Torre

## Description of the data

### What type of data do you have?

>The chosen dataset is recent-grads.csv. The data is sourced from the American Community Survey 2010-2012 public use microdata series and contains information on basic earnings and labor force information associated to various college majors. Most features are numerical values of employment status by major, additionally containing features such as sex and type of job. The data is taken from recent graduates:

| Variable Name | Description| Data type |
| :-- | :-- | :-- |
| Major_code |Major code, FO1DP in ACS PUMS|Identifier of the row
| Major | Major description| Descriptive
| Major_category| Category of major from Carnevale et al| Categorical
| Total| Total number of people with major| Numerical (count)
| Men| Number of men in major| Numerical (count)
| Women| Number of women in major| Numerical (count)
| Employed| Number employed (ESR == 1 or 2)| Numerical (count)
| Unemployed| Number unemployed (ESR == 3)| Numerical (count)
| Full_time_year_round | Employed at least 50 weeks (WKW == 1) and at least 35 hours (WKHP >= 35)|Numerical (count)
| Median_income| Median earnings of full-time, year-round workers| Numerical (real)
| College_jobs| Number with job requiring a college degree| Numerical (count)
| Non_college_jobs| Number with job not requiring a college degree| Numerical (count)
| Low_wage_jobs| Number in low-wage service jobs| Numerical (count)


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython import display
import altair as alt 
alt.data_transformers.enable('json')

recent_grads = pd.read_csv('data/recent-grads.csv')

to_drop = ['Rank','Full_time','Part_time', 'Unemployment_rate','ShareWomen', 'P25th','P75th',
           'Sample_size']
recent_grads.drop(to_drop, axis=1, inplace=True)
recent_grads['Major_code'] = recent_grads['Major_code'].apply(str)

recent_grads.dropna(subset = ['Total'], inplace=True)
recent_grads.isna().sum()
recent_grads.astype({'Total': 'int64', 'Men': 'int64', 'Women': 'int64'}).dtypes

recent_grads = recent_grads.rename(columns={'Median': 'Median_income'})

# Convert major name to title case
recent_grads['Major'] = recent_grads['Major'].str.title()

# Get short names of majors
major_list = pd.read_csv('data/major_short_names.csv')
recent_grads = pd.merge(major_list, recent_grads, how='right')
recent_grads.rename(columns={'Major':'Major_long'}, inplace=True)
recent_grads.rename(columns={'Major_short':'Major'}, inplace=True)

In [ ]:
recent_grads.head()

### What is the range of values? Which units are used? What precision is required?

>The numerical data in the given datasets are represented by counts and rates, aside from 'Median' (US Dollars). The table below describes key statistical features for each relavent column, including a minimum and maximum. All columns aside from 'Median' represent individual persons and therefore the precision should remain as whole integers.

In [ ]:
recent_grads.describe()

### What is it's life span? (how often should it be updated)

>The dataset is taken from census surveys from 2010-2012. It is reasonable to assume a short lifespan considering the constant flux of the labor market, and thereby should ideally be updated each year to maintain accuracy.

### How is it distributed?

>The first figure shows representation of major categories, where areas such as engineering and education have much higher occurence than communications and interdisciplinary majors shown left. The mean Median income for major categories is shown on right. The figure 'Distributions of Numerical Variables' plots histograms for each variables, revealing left-skews for all variables aside from 'Median' which takes on a more normal distribution.

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20,6))
plt.suptitle('Major Categories: Occurence & Income', fontsize=18)

recent_grads['Major_category'].value_counts().plot(ax=ax1, kind='bar', color='tab:blue'
                                                  ).set_title('Occurence')
#.set_title('Major-Category Occurence', fontsize=16)

sal_groups = recent_grads[['Major_category','Median_income']].groupby('Major_category').mean()
sal_groups.sort_values(by=['Median_income'], ascending=False).plot(ax=ax2, kind='bar', color='tab:green'
                                                           ).set_title('Median Income')


In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9)) = plt.subplots(nrows=3, ncols=3, figsize = (18,14))
plt.suptitle('Distributions of Numerical Variables', fontsize=18)

recent_grads['Total'].plot(ax=ax1, kind='hist', bins=50, color='tab:green').set_title('Total')
recent_grads['Men'].plot(ax=ax2, kind='hist', bins=50, color='tab:red').set_title('Men')
recent_grads['Women'].plot(ax=ax3, kind='hist', bins=50,color='tab:blue').set_title('Women')
recent_grads['Employed'].plot(ax=ax4, kind='hist', bins=50,color='tab:green').set_title('Employed')
recent_grads['Unemployed'].plot(ax=ax5, kind='hist', bins=50,color='tab:red').set_title('Unemployed')
recent_grads['Median_income'].plot(ax=ax6, kind='hist', bins=50,color ='tab:blue').set_title('Median_income')
recent_grads['College_jobs'].plot(ax=ax7, kind='hist', bins=50,color ='tab:green').set_title('College_jobs')
recent_grads['Non_college_jobs'].plot(ax=ax8, kind='hist', bins=50,color ='tab:red').set_title('Non_college_jobs')
recent_grads['Low_wage_jobs'].plot(ax=ax9, kind='hist', bins=50,color ='tab:blue').set_title('Low_wage_jobs')


### Are there outliers? Identify relations and groupings

>The prominent grouping of this dataset is by 'Major_category' to compare records of the same fields internally and against other groups. Looking at the box plots of numerical variables by Major category leads some initial intuition of outliers. Varables 'Total', 'Women', 'Unemployed', and 'Non_college_jobs' have obvious outliers.

In [ ]:
recent_grads[3:].plot(x='Major_category',kind='box', figsize=(15,10))
plt.title('Variable Distributions by Major Category', fontsize=18)

# Audience

The audience of these charts are not expected to be experts in the usage of charts. They will access the charts from a PC or a smartphone. They will consult the visualization occasionally.

The main **message** of these charts is that the kind of job that graduates may obtain depends strongly on their major.
An additional message is that there may be differences between male and female students in their choice of majors.

We have identified the following typical kinds of users ("**personas**"):

- Persona 1: Prospective student  
  Goal: Help them choose a major, by understanding what impact of their choice may have in their future job opportunities and   earnings.  

- Persona 2: Faculty administration  
  Goal: Understand which majors are more demanded by students and by the job market, and take it into account for planning the offering of places and hiring of new professors.  

- Persona 3: Government official  
  Goal: Investigate possible actions to improve social mobility, by helping children of poor families to obtain a degree that may lead to well-paid jobs. Investigate majors in which the rate of female students is very low, to possibly promote actions against negative stereotypes. Investigate which majors typically result in jobs requiring a college degree or not.  

Based on the goals indicated previously, here are a number of **questions** that the charts should help to answer.
- How do median earnings of recent graduates vary depending on their kind of major?
- Which are the majors with the highest earnings?
- Which are the majors with the most graduate students?
- Is there a difference on the chosen majors between male and female students?
- Which majors/major categories are more likely to result in a job requiring a college degree?


## Scenario

Maria has just finished high school and is wondering what degree she should study. In principle she is interested in Mathematics and also in Psychology.

Maria hears about the dashboard that compares job opportunities for graduates of different majors, and she decides to consult it. 

In the top-level dashboard she sees that the area of Mathematics offers much better chances to get a job than Psychology, as well as considerably higher salaries. Within Mathematics, she checks the drill-down by specific major and realises that the major of Applied Mathematics, which she is attracted to, offers good prospects to obtain qualified jobs (that require a degree) as well as an outstanding expected income. This helps her decide to major in this subject.

# "Napkin" design

### Dashboard level 1

Level 1 shows data by **major category**. For each category, the **average income** and the **% of employed** students are displayed in two parallel bar charts.

A tooltip is available to view the exact data.

In [ ]:
display.Image("./images/napkin_design_level1.png")

### Dashboard level 2

By clicking on the bar of a major category, we drill down into level 2.

Level 2 shows data by **major**, filtered by the selected major category.

For each major, the **average income** and the **% of employed** students are displayed in two parallel bar charts. The % of employment is divided between jobs that **require a degree** and those that don't. We can see as context the average values for the category and overall.

A tooltip is available to view the exact data.

In [ ]:
display.Image("./images/napkin_design_level2.png")

# Task 4: First Implementations in Altair

**Note on the calculation of the median income:** We have the median income for each major, and need to aggregate it into major categories. Strictly speaking, it is not possible to infer the real median income by category from this data. We will approximate it using the mean of the different median incomes weighted by the number of students in each major.

**Issue:** college + non_college jobs do not add up to "employed". We wondered if the "low wage jobs" would be a separate category that had to be added to calculate the % of college jobs. This is not the case.
Finally, to calculate % of college jobs, we just do `college_jobs/(college_jobs+non_college_jobs)`

In [ ]:
recent_grads['Percent_employed'] = recent_grads['Employed'] / (recent_grads['Employed'] + recent_grads['Unemployed'])
recent_grads['Percent_college_jobs'] = recent_grads['College_jobs'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'])
recent_grads['Percent_non_college_jobs'] = recent_grads['Non_college_jobs'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'])
# recent_grads['Percent_low_wage_jobs'] = recent_grads['Low_wage_jobs'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'] + recent_grads['Low_wage_jobs'])
recent_grads['Total_income'] = recent_grads['Median_income'] * recent_grads['Full_time_year_round']
recent_grads.head()

In [ ]:
# sum_cols = ['Total', 'Men', 'Women', 'Employed', 'Unemployed', 'College_jobs', 'Non_college_jobs', 'Low_wage_jobs']

df_cat = recent_grads.groupby('Major_category').agg({'Total_income': 'sum', 'Total': 'sum', 'Men': 'sum', 'Women': 'sum',
                    'Employed': 'sum', 'Unemployed': 'sum', 'Full_time_year_round': 'sum', 'College_jobs': 'sum',
                    'Non_college_jobs': 'sum', 'Low_wage_jobs': 'sum'})
df_cat.reset_index(level=0, inplace=True)

# See note above about the approximation of median income by category
df_cat['Median_income'] = round(df_cat['Total_income'] / df_cat['Full_time_year_round'])
df_cat['Percent_employed'] = (df_cat['Employed'] / (df_cat['Employed'] + df_cat['Unemployed']))*100
df_cat['Percent_unemployed'] = (df_cat['Unemployed'] / (df_cat['Employed'] + df_cat['Unemployed']))*100
df_cat['Percent_men'] = (df_cat['Men'] / (df_cat['Men'] + df_cat['Women']))*100
df_cat['Percent_women'] = (df_cat['Women'] / (df_cat['Men'] + df_cat['Women']))*100
df_cat['Percent_college_jobs'] = (df_cat['College_jobs'] / (df_cat['College_jobs'] + df_cat['Non_college_jobs']))*100
df_cat['Percent_non_college_jobs'] = (df_cat['Non_college_jobs'] / (df_cat['College_jobs'] + df_cat['Non_college_jobs']))*100
# df_cat['Percent_low_wage_jobs'] = df_cat['Low_wage_jobs'] / (df_cat['College_jobs'] + df_cat['Non_college_jobs'] + df_cat['Low_wage_jobs'])
df_cat.head()

In [ ]:
# Required for stacked bar chart: Data set with separate rows for "Percent_employed" and "Percent_unemployed"
df_cat_E = pd.melt(df_cat, id_vars=['Major_category', 'Median_income'], value_vars=['Percent_employed', 'Percent_unemployed'], 
        var_name='Status', value_name='Percentage')
df_cat_E.head()

In [ ]:
# Major categories sorted by median income - used to keep the same sorting as the other chart
sorted_categories = df_cat.sort_values(by=['Median_income'], ascending=False)['Major_category'].tolist()

### Level 1 Visualizations

In [ ]:

#selection = alt.selection_single();
#color="#73a1eb"

def cat_plots():
    bars = alt.Chart(df_cat).mark_bar(color='#87CEFA', size=20, opacity=0.8).encode(
        y = alt.Y('Major_category:N', title=None, sort= '-x', axis=alt.Axis(labels=False, tickSize=0)),
        x = alt.X('Median_income:Q', title=None),
#        color = alt.condition(selection, 'Major_category:N', alt.value('grey')),
        tooltip = [alt.Tooltip('Major_category', title='Major Category'),
                   alt.Tooltip('Median_income', title='Median Income'),
                   alt.Tooltip('Total', title='Total Students'),
    ])

    color = 'crimson'
        
    rule = alt.Chart(recent_grads).mark_rule(color=color).encode(
        x = alt.X('mean(Median_income):Q'),
        size = alt.value(3),
#        tooltip = [alt.Tooltip('mean(Global Average')]
    )
    
    fontSize = 16
    dy = -80
    angle = 90
    fontWeight='bold'
    
    text1 = alt.Chart(recent_grads).mark_text(dx=300, dy=dy, color=color, fontSize=fontSize, text='Global Average: $', angle=angle, fontWeight=fontWeight).encode(
        y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    )
    
    text2 = alt.Chart(recent_grads).mark_text(dx=390, dy=dy, color=color, fontSize=fontSize, angle=angle, fontWeight=fontWeight).encode(
        text = alt.Text('mean(Median_income):Q', format='.0f'),
        y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    )

    incomes = (bars + rule + text1 + text2).properties(
        width=400,
        height=550,
        title='Median Income (USD)'
    )    


    # Bar chart showing percent employed by category
    employments = alt.Chart(df_cat_E).mark_bar(size=20, opacity=0.95).encode(
        y= alt.Y('Major_category:N', title=None, sort=sorted_categories, axis=alt.Axis(tickSize=0)),
        x= alt.X('Percentage:Q', title=None),
        color= alt.Color('Status:N', scale = alt.Scale(domain=['Percent_employed', 'Percent_unemployed'], range=['royalblue', 'lightgrey'])),
        order=alt.Order('Status', sort='ascending')
    ).properties(
        width=400,
        height=550,
        title='Employment Status (%)'
    )

    return alt.hconcat(employments, incomes).properties(spacing=5).configure_title(fontSize=18).configure_axis(
        grid=False,
        labelFontSize=14,
        titleFontSize=14
    )

dashboard = cat_plots()
dashboard

In [ ]:
import json
from pathlib import Path

Path('dashboard-spec.json').write_text(json.dumps(dashboard.to_dict()))

- **PENDING in the chart above**: Label the red line as "Global average"

- **PENDING in the chart above**: Tooltips

### Level 2 Visualizations

For the second bar chart, calculate percent of employed and unemployed.
The data on college and non-college jobs does not add up to the total of jobs. This must be due to missing information, i.e. some employed graduates did not report this information. We will extrapolate and apply the % of college jobs to all jobs.
Therefore, the total number of students will be split into three separate groups:
- Unemployed
- Employed in job not requiring college
- Employed in job requiring college


In [ ]:
recent_grads['Percent_unemployed'] = recent_grads['Unemployed'] / (recent_grads['Employed'] + recent_grads['Unemployed'])
recent_grads['Percent_employed'] = recent_grads['Employed'] / (recent_grads['Employed'] + recent_grads['Unemployed'])

# Split employed between college and non-college
recent_grads['Percent_employed_college_jobs'] = recent_grads['Percent_employed']*recent_grads['College_jobs'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'])
recent_grads['Percent_employed_non_college_jobs'] = recent_grads['Percent_employed']*recent_grads['Non_college_jobs'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'])
recent_grads.head()

# Required for stacked bar chart: Data set with separate rows for 'Percent_unemployed', 'Percent_employed_college_jobs' and
# 'Percent_employed_non_college_jobs'

df_RGE = pd.melt(recent_grads, id_vars=['Major', 'Major_category'], 
              value_vars=['Percent_unemployed', 'Percent_employed_college_jobs', 'Percent_employed_non_college_jobs'],
        var_name='Status', value_name='Percentage')
df_RGE.head()

# Majors sorted by median income - used to keep the same sorting as the other chart
sorted_majors = recent_grads.sort_values(by=['Median_income','Major'], ascending=[False, True])['Major'].tolist()

In [ ]:
recent_grads[recent_grads.Major_category=='Engineering'].Major.count()

In [ ]:
def major_plots(major_category):
    """plots income/employment charts for selected major"""
    
    num_majors = recent_grads[recent_grads.Major_category == major_category].Major.count()
    width = 500
    height = num_majors*25
    
    # plot income by major

    bars = alt.Chart(recent_grads).mark_bar(color='#87CEFA', size=15, opacity=0.8).encode(
        y = alt.Y('Major:N', title=None, sort= ['-x','Major'], axis=alt.Axis(labels=False, tickSize=0)),
        x = alt.X('Median_income:Q', title=None, scale=alt.Scale(domain=[0, 110000])),
        tooltip = [alt.Tooltip('Major', title='Major'),
                   alt.Tooltip('Percent_employed', title='Percent Employed'),
                   alt.Tooltip('Percent_college_jobs', title='Percent College Jobs'),
    ]).transform_filter(
        (alt.datum.Major_category == major_category)
    )
    
    # average lines and labels
    
    color1 = 'crimson'
    color2 = 'dimgray'

    rule_cat = alt.Chart(recent_grads).mark_rule(color=color2).encode(
        x = alt.X('mean(Median_income):Q'),
        size = alt.value(3)
    ).transform_filter(
        (alt.datum.Major_category == major_category)
    )

    rule_global = alt.Chart(recent_grads).mark_rule(color=color1).encode(
        x = alt.X('mean(Median_income):Q'),
        size = alt.value(3)
    )
    
    fontSize = 14
    fontWeight = 'normal'
    
    annotation_locs = {'Agriculture & Natural Resources': (100, 165, 170, 60, 80, 90),
                   'Arts': (75, 140, 160, 60, 90, 90),
                   'Biology & Life Science': (150, 215, 235, 60, 80, 90),
                   'Business': (120, 185, 185, 270, 250, 0),
                   'Communications & Journalism': (120, 185, 185, 70, 50, 0),
                   'Computers & Mathematics': (120, 185, 185, 230, 210, 0),
                   'Education': (250, 315, 335, 60, 95, 90),
                   'Engineering': (400, 465, 525, 60, -20, 90),
                   'Health': (120, 185, 185, 250, 230, 0),
                   'Humanities & Liberal Arts': (200, 265, 285, 60, 95, 90),
                   'Industrial Arts & Consumer Services': (120, 185, 185, 140, 120, 0),
                   'Interdisciplinary': (120, 185, 185, 90, 70, 0),
                   'Law & Public Policy': (120, 185, 185, 100, 80, 0),
                   'Physical Sciences': (120, 185, 185, 200, 180, 0),
                   'Psychology & Social Work': (120, 185, 185, 190, 170, 0),
                   'Social Science': (120, 185, 185, 190, 170, 0)
                  }
    
    dx_text, dx_val1, dx_val2, dy1, dy2, angle = annotation_locs[major_category]
    

    text1 = alt.Chart(recent_grads).mark_text(dx=dx_text, dy=dy1, color=color1, fontSize=fontSize, fontWeight=fontWeight, text='Global Avg: $', angle=angle).encode(
        y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    )
    
    text2 = alt.Chart(recent_grads).mark_text(dx=dx_val1, dy=dy1, color=color1, fontSize=fontSize, fontWeight=fontWeight,angle=angle).encode(
        text = alt.Text('mean(Median_income):Q', format='.0f'),
        y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    )
    
    text3 = alt.Chart(recent_grads).mark_text(dx=dx_text, dy=dy2, color=color2, fontSize=fontSize, fontWeight=fontWeight, text='Category Avg: $', angle=angle).encode(
        y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    )
    
    text4 = alt.Chart(recent_grads).mark_text(dx=dx_val2, dy=dy2, color=color2, fontSize=fontSize, fontWeight=fontWeight, angle=angle).encode(
        text = alt.Text('mean(Median_income):Q', format='.0f'),
        y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    ).transform_filter(
        (alt.datum.Major_category == major_category)
    )
    
    income = (bars + rule_cat + rule_global + text1 + text2 + text3 + text4).properties(
        width=width,
        height=height,
        title='Median Income (USD)')
    
    # plot employment status by major 
    
    employment = alt.Chart(df_RGE).mark_bar(size=15).encode(
        y = alt.Y('Major:N', title=None, sort=sorted_majors, axis=alt.Axis(labelPadding=15, tickSize=0, labelFontSize=11)),
        x = alt.X('Percentage:Q', scale=alt.Scale(domain=[0, 1.0]), title=None),
        color = alt.Color('Status:N', 
                          scale = alt.Scale(domain=['Percent_employed_college_jobs', 'Percent_employed_non_college_jobs','Percent_unemployed'], range=['royalblue', 'darkgrey', 'lightgrey'])),
        order = alt.Order('Status', sort='ascending')
    ).properties(
        width=width,
        height=height,
        title='Employment Status (%)'
    ).transform_filter(
        (alt.datum.Major_category == major_category)
    )

    return alt.hconcat(employment, income).properties(spacing=5).configure_title(fontSize=18).configure_axis(
        grid=False,
        labelFontSize=14,
        titleFontSize=14
    )

major_plots('Engineering')

## Testing dynamic filtering

In [ ]:
select_box = alt.binding_select(options=sorted_categories, name='major_category')
sel = alt.selection_single(fields=['major_category'], bind=select_box, init={'major_category': 'Agriculture & Natural Resources'})



num_majors = recent_grads[recent_grads.Major_category == major_category].Major.count()
width = 500
height = num_majors*25

# plot income by major

bars = alt.Chart(recent_grads).mark_bar(color='#87CEFA', size=15, opacity=0.8).encode(
    y = alt.Y('Major:N', title=None, sort= ['-x','Major'], axis=alt.Axis(labels=False, tickSize=0)),
    x = alt.X('Median_income:Q', title=None, scale=alt.Scale(domain=[0, 110000])),
    tooltip = [alt.Tooltip('Major', title='Major'),
               alt.Tooltip('Percent_employed', title='Percent Employed'),
               alt.Tooltip('Percent_college_jobs', title='Percent College Jobs'),
]).transform_filter(
    (alt.datum.Major_category == major_category)
)

# average lines and labels

color1 = 'crimson'
color2 = 'dimgray'

rule_cat = alt.Chart(recent_grads).mark_rule(color=color2).encode(
    x = alt.X('mean(Median_income):Q'),
    size = alt.value(3)
).transform_filter(
    (alt.datum.Major_category == major_category)
)

rule_global = alt.Chart(recent_grads).mark_rule(color=color1).encode(
    x = alt.X('mean(Median_income):Q'),
    size = alt.value(3)
)

fontSize = 14
fontWeight = 'normal'

annotation_locs = {'Agriculture & Natural Resources': (100, 165, 170, 60, 80, 90),
               'Arts': (75, 140, 160, 60, 90, 90),
               'Biology & Life Science': (150, 215, 235, 60, 80, 90),
               'Business': (120, 185, 185, 270, 250, 0),
               'Communications & Journalism': (120, 185, 185, 70, 50, 0),
               'Computers & Mathematics': (120, 185, 185, 230, 210, 0),
               'Education': (250, 315, 335, 60, 95, 90),
               'Engineering': (400, 465, 525, 60, -20, 90),
               'Health': (120, 185, 185, 250, 230, 0),
               'Humanities & Liberal Arts': (200, 265, 285, 60, 95, 90),
               'Industrial Arts & Consumer Services': (120, 185, 185, 140, 120, 0),
               'Interdisciplinary': (120, 185, 185, 90, 70, 0),
               'Law & Public Policy': (120, 185, 185, 100, 80, 0),
               'Physical Sciences': (120, 185, 185, 200, 180, 0),
               'Psychology & Social Work': (120, 185, 185, 190, 170, 0),
               'Social Science': (120, 185, 185, 190, 170, 0)
              }

dx_text, dx_val1, dx_val2, dy1, dy2, angle = annotation_locs[major_category]


text1 = alt.Chart(recent_grads).mark_text(dx=dx_text, dy=dy1, color=color1, fontSize=fontSize, fontWeight=fontWeight, text='Global Avg: $', angle=angle).encode(
    y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
)

text2 = alt.Chart(recent_grads).mark_text(dx=dx_val1, dy=dy1, color=color1, fontSize=fontSize, fontWeight=fontWeight,angle=angle).encode(
    text = alt.Text('mean(Median_income):Q', format='.0f'),
    y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
)

text3 = alt.Chart(recent_grads).mark_text(dx=dx_text, dy=dy2, color=color2, fontSize=fontSize, fontWeight=fontWeight, text='Category Avg: $', angle=angle).encode(
    y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
)

text4 = alt.Chart(recent_grads).mark_text(dx=dx_val2, dy=dy2, color=color2, fontSize=fontSize, fontWeight=fontWeight, angle=angle).encode(
    text = alt.Text('mean(Median_income):Q', format='.0f'),
    y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
).transform_filter(
    (alt.datum.Major_category == major_category)
)

income = (bars + rule_cat + rule_global + text1 + text2 + text3 + text4).properties(
    width=width,
    height=height,
    title='Median Income (USD)')

# plot employment status by major 

employment = alt.Chart(df_RGE).mark_bar(size=15).encode(
    y = alt.Y('Major:N', title=None, sort=sorted_majors, axis=alt.Axis(labelPadding=15, tickSize=0, labelFontSize=11)),
    x = alt.X('Percentage:Q', scale=alt.Scale(domain=[0, 1.0]), title=None),
    color = alt.Color('Status:N', 
                      scale = alt.Scale(domain=['Percent_employed_college_jobs', 'Percent_employed_non_college_jobs','Percent_unemployed'], range=['royalblue', 'darkgrey', 'lightgrey'])),
    order = alt.Order('Status', sort='ascending')
).properties(
    width=width,
    height=height,
    title='Employment Status (%)'
).transform_filter(
    (alt.datum.Major_category == major_category)
)

return alt.hconcat(employment, income).properties(spacing=5).configure_title(fontSize=18).configure_axis(
    grid=False,
    labelFontSize=14,
    titleFontSize=14
)

In [ ]:
major_cats_list = df_cat.Major_category.to_list()
select_box = alt.binding_select(options=major_cats_list, name='major_category')
sel = alt.selection_single(fields=['major_category'], bind=select_box, init={'major_category': 'Agriculture & Natural Resources'})


# plot income by major

bars = alt.Chart(recent_grads).mark_bar(color='#87CEFA', size=15, opacity=0.8).encode(
    y = alt.Y('Major:N', title=None, sort= ['-x','Major'], axis=alt.Axis(labels=False, tickSize=0)),
    x = alt.X('Median_income:Q', title=None, scale=alt.Scale(domain=[0, 110000])),
    tooltip = [alt.Tooltip('Major', title='Major'),
               alt.Tooltip('Percent_employed', title='Percent Employed'),
               alt.Tooltip('Percent_college_jobs', title='Percent College Jobs'),
# ]).transform_fold(
#     sorted_categories,
#     as_=['column', 'value']
]).transform_filter(
    sel
).add_selection(
    sel
)

bars

In [ ]:
def major_plots(major_category):
    """plots income/employment charts for selected major"""
    
    num_majors = recent_grads[recent_grads.Major_category == major_category].Major.count()
    width = 400
    height = num_majors*25
    
    
    # plot income by major

    bars = alt.Chart(recent_grads).mark_bar(color='#87CEFA', size=15, opacity=0.8).encode(
        y = alt.Y('Major:N', title=None, sort= ['-x','Major'], axis=alt.Axis(labels=False, tickSize=0)),
        x = alt.X('Median_income:Q', title=None, scale=alt.Scale(domain=[0, 110000])),
        tooltip = [alt.Tooltip('Major', title='Major'),
                   alt.Tooltip('Percent_employed', title='Percent Employed'),
                   alt.Tooltip('Percent_college_jobs', title='Percent College Jobs'),
    ]).transform_filter(
        (alt.datum.Major_category == major_category)
    )
    
    
    # average lines and labels
    
    color1 = 'crimson'
    color2 = 'dimgray'
    

    rule_cat = alt.Chart(recent_grads).mark_rule(color=color2).encode(
        x = alt.X('mean(Median_income):Q'),
        size = alt.value(3)
    ).transform_filter(
        (alt.datum.Major_category == major_category)
    )
    

    rule_global = alt.Chart(recent_grads).mark_rule(color=color1).encode(
        x = alt.X('mean(Median_income):Q'),
        size = alt.value(3)
    )
    
    fontSize = 14
    fontWeight = 'normal'
    
    dx_text = 85
    dx_val1 = dx_text+60
    dx_val2 = dx_text+70
    dy1 = height-100 #set for global average
    dy2 = height-120
    

    text1 = alt.Chart(recent_grads).mark_text(dx=dx_text, dy=dy1, color=color1, fontSize=fontSize, fontWeight=fontWeight, text='Global Avg: $').encode(
                y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    )
    
    text2 = alt.Chart(recent_grads).mark_text(dx=dx_val1, dy=dy1, color=color1, fontSize=fontSize, fontWeight=fontWeight).encode(
        text = alt.Text('mean(Median_income):Q', format='.0f'),
        y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    )
    
    text3 = alt.Chart(recent_grads).mark_text(dx=dx_text, dy=dy2, color=color2, fontSize=fontSize, fontWeight=fontWeight, text='Category Avg: $').encode(
        y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    )
    
    text4 = alt.Chart(recent_grads).mark_text(dx=dx_val2, dy=dy2+48, color=color2, fontSize=fontSize, fontWeight=fontWeight).encode(
        text = alt.Text('mean(Median_income):Q', format='.0f'),
        y= alt.Y('mean(Median_income)', axis=alt.Axis(labels=False, tickSize=0), title=None)
    ).transform_filter(
        (alt.datum.Major_category == major_category)
    )
    
    income = (bars + rule_cat + rule_global + text1 + text2 + text3 + text4).properties(
        width=width,
        height=height,
        title='Median Income (USD)')
    
    
    # plot employment status by major 
    
    employment = alt.Chart(df_RGE).mark_bar(size=15).encode(
        y = alt.Y('Major:N', title=None, sort=sorted_majors, axis=alt.Axis(tickSize=0, labelFontSize=11, labelPadding=10)),
        x = alt.X('Percentage:Q', scale=alt.Scale(domain=[0, 1.0]), title=None),
        color = alt.Color('Status:N', 
                          scale = alt.Scale(domain=['Percent_employed_college_jobs', 'Percent_employed_non_college_jobs','Percent_unemployed'], range=['royalblue', 'darkgrey', 'lightgrey'])),
        order = alt.Order('Status', sort='ascending')
    ).properties(
        width=width,
        height=height,
        title='Employment Status (%)'
    ).transform_filter(
        (alt.datum.Major_category == major_category)
    )

    return alt.hconcat(employment, income).properties(spacing=5).configure_title(fontSize=18).configure_axis(
        grid=False,
        labelFontSize=14,
        titleFontSize=14
    )

major_plots('Engineering')

In [ ]:
# {'category': (dx_text, dy2)}

annotation_locs = {'Agriculture & Natural Resources': (),
                   'Arts': (),
                   'Biology & Life Science': (),
                   'Business': (),
                   'Communications & Journalism': (),
                   'Computers & Mathematics': (),
                   'Education': (),
                   'Engineering': (400, -20),
                   'Health': (),
                   'Humanities & Liberal Arts': (),
                   'Industrial Arts & Consumer Services': (),
                   'Interdisciplinary': (),
                   'Law & Public Policy': (),
                   'Physical Sciences': (),
                   'Psychology & Social Work': (),
                   'Social Science': ()
                  }

In [ ]:
# See list of categories
for i, major_cat in enumerate(major_cats_list):
    print(i, major_cat)

In [ ]:
# Generate JSON file for every category
for i, major_cat in enumerate(major_cats_list):
    dashboard = major_plots(major_cat)
    Path('dashboard-cat' + str(i) + '-spec.json').write_text(json.dumps(dashboard.to_dict()))


In [ ]:
def supp_charts(major_category, category_list):
    base = alt.Chart(recent_grads).mark_bar(color='#87CEFA', size=15).encode(
        x = alt.X('Major:N', title=None, sort= ['-y','Major'], axis=alt.Axis(labels=False)),
        y = alt.Y('Median_income:Q', title='Mean Income'),
        tooltip = [alt.Tooltip('Major', title='Major'),
                   alt.Tooltip('Percent_employed', title='Percent Employed'),
                   alt.Tooltip('Percent_college_jobs', title='Percent College Jobs'),
    ])

    rule_cat = alt.Chart(recent_grads).mark_rule(color='grey').encode(
        y = alt.Y('mean(Median_income):Q'),
        size = alt.value(3)
    )

    rule_global = alt.Chart(recent_grads).mark_rule(color='red').encode(
        y = alt.Y('mean(Median_income):Q'),
        size = alt.value(3)
    )

    income = (base + rule_cat + rule_global).properties(
        width=800,
        height=400,
        title='Median Income')

    chart = alt.hconcat()
    for major in major_cats_list:
        chart |= base.transform_filter(alt.datum.Major_category == major)
    chart

In [ ]:
base = alt.Chart(recent_grads).mark_bar(color='#203082', size=15).encode(
    x = alt.X('Major:N', title=None, sort= ['-y','Major'], axis=alt.Axis(labels=False)),
    y = alt.Y('Median_income:Q', title='Mean Income'),
    tooltip = [alt.Tooltip('Major', title='Major'),
               alt.Tooltip('Percent_employed', title='Percent Employed'),
               alt.Tooltip('Percent_college_jobs', title='Percent College Jobs'),
])

rule_cat = alt.Chart(recent_grads).mark_rule(color='grey').encode(
    y = alt.Y('mean(Median_income):Q'),
    size = alt.value(3)
)

rule_global = alt.Chart(recent_grads).mark_rule(color='red').encode(
    y = alt.Y('mean(Median_income):Q'),
    size = alt.value(3)
)

income = (base + rule_cat + rule_global).properties(
    width=800,
    height=400,
    title='Median Income')

chart = alt.hconcat()
for major in major_cats_list:
    chart |= base.transform_filter(alt.datum.Major_category == major)
chart

- **PENDING in the chart above**: Label the red lines as "Engineering average" and "Global average"

- **PENDING in the chart above**: Tooltips

# Task 5: Refine charts taking into account perception

### Income

The most important variable is the **income**. This is displayed with vertical bars using bar length.

Advantages:
- Type of chart is well known and understood
- Easy to perceive: **length** is a powerful preattentive property.
- Easy to compare between bars (majors and major categories)

### % Employment, % Employment college jobs
Other variables to be shown and compared are "**% Employment**" and "**% Employment college jobs**".
For these, stacked bars have been selected, with each bar adding up to 100%.

Advantages:
- Type of chart is well known and understood
- Easy to perceive: **length** is a powerful preattentive property. Besides, the different sections of the bars are differenciated with **color**, with is also preattentive. The colors (blue, orange, red) are from Altair's default palette and are accessible for color-blind people.
- Easy to compare between bars (majors and major categories)

Potential disadvantage:
- The bar sections that do not start at the bottom cannot be compared visually. This means that the percent of unemployment and percent of non-college jobs are not easy to compare.   
This is however considered a minor issue, as the users are mostly interested in comparing Percent of employment and Percent of jobs requiring college, and these are visually comparable. In addition, the tooltip shows the numeric values if needed.

### Context: Global and category average
In order to provide context, the global average and the average for the category are shown as horizontal lines. This relies on the preattentive property of position.


In [ ]:
alt.vconcat(cat_plots, major_plots('Engineering')).properties(spacing=5).configure_title(fontSize=18).save('all_dash.html')